### Excercise for season 4 of the DS4A course:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

dir_data = "C:/Users/Acer/Documents/DSA4/ClassesNotebooks/DS4A_prg/cases/Cases_JupyterNote/case_1.4_student/"     

In [2]:
# Reading the data
dir_file = dir_data + 'EnergySectorData.csv'
raw_df = pd.read_csv(dir_file)
raw_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D


In [3]:
# Checking missing data:
raw_df.isnull().sum()

Date          0
Open          2
High         14
Low           7
Close         7
Adj Close     7
Volume       22
Symbol        0
dtype: int64

In [4]:
# % of missing values
raw_df.isnull().mean() *100

Date         0.000000
Open         0.031873
High         0.223108
Low          0.111554
Close        0.111554
Adj Close    0.111554
Volume       0.350598
Symbol       0.000000
dtype: float64

In [5]:
# Removing missing data
progress_df = raw_df.dropna().copy()

## Standarazing dates

In [6]:
progress_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1230,1230,1230,1230,1230,1230,1230
DUK,1249,1249,1249,1249,1249,1249,1249
EXC,1239,1239,1239,1239,1239,1239,1239
NEE,1251,1251,1251,1251,1251,1251,1251
SO,1259,1259,1259,1259,1259,1259,1259


Let's create a new clean_df that corresponds to a DataFrame with the same number of rows for each Symbol, where all symbols share the same set of dates:

In [7]:
# Unique date values per symbol:

set_dates_D = set(progress_df[progress_df['Symbol'] == 'D']['Date'])
set_dates_D = set(progress_df[progress_df['Symbol'] == 'D']['Date'])
set_dates_EXC = set(progress_df[progress_df['Symbol'] == 'EXC']['Date'])
set_dates_NEE = set(progress_df[progress_df['Symbol'] == 'NEE']['Date'])
set_dates_SO = set(progress_df[progress_df['Symbol'] == 'SO']['Date'])
set_dates_DUK = set(progress_df[progress_df['Symbol'] == 'DUK']['Date'])

# Intersection of all symbols:

set_unique_dates = set.intersection(set_dates_D,set_dates_EXC,set_dates_NEE,set_dates_SO,set_dates_DUK)


In [8]:
# Filter new DataFrame for only the dates that are present in every symbol (i.e. the overlapping dates)
clean_df = progress_df[progress_df['Date'].isin(set_unique_dates)].copy()

In [9]:
# Let's take a look
clean_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1192,1192,1192,1192,1192,1192,1192
DUK,1192,1192,1192,1192,1192,1192,1192
EXC,1192,1192,1192,1192,1192,1192,1192
NEE,1192,1192,1192,1192,1192,1192,1192
SO,1192,1192,1192,1192,1192,1192,1192


### Exercise 1:
Write code to ensure that each of the symbols share the same set of unique dates. (Hint: use the ```set()``` method.)

In [10]:
# Unique list of symbols

symbol_list = clean_df.Symbol.unique()

# Comparing each symbol

for i in range(len(symbol_list)-1):
    
    # Unique dates
    u_d_s = set(clean_df.loc[clean_df['Symbol'] == symbol_list[i],'Date'])
    # Next dates
    u_nx_s = set(clean_df.loc[clean_df['Symbol'] == symbol_list[i+1],'Date'])
    # Conditinal
    if u_d_s == u_nx_s:
        print("Symbol: ",symbol_list[i],"and Symbol: ",symbol_list[i+1] ,", have the same dates")
    else: 
        rint("Symbol: ",symbol_list[i],"and Symbol: ",symbol_list[i+1] ,", dont have the same dates")

Symbol:  D and Symbol:  EXC , have the same dates
Symbol:  EXC and Symbol:  NEE , have the same dates
Symbol:  NEE and Symbol:  SO , have the same dates
Symbol:  SO and Symbol:  DUK , have the same dates


Recall that the original question requires us to investigate both daily stock returns and the volatility of those returns. This means that important measures of interest are:

1. Daily (open to close) stock return
2. Volatility of daily stock return

Why are each of these important?

1. Volatility: Gives insight into amount of price movement in any given day. Volatility is directly related to the level of risk involved in holding the stock.
2. Return: Gives us an idea of the return on investment over a period of time.

Let's calculate these statistics and add them to the DataFrame ```clean_df```:

In [11]:
clean_df['VolStat'] = (clean_df['High'] - clean_df['Low']) / clean_df['Open']
clean_df['Return'] = (clean_df['Close'] / clean_df['Open']) - 1.0
clean_df['Volume_Millions'] = clean_df['Volume'] / 1000000.0 # Volume in Millions (added for convenience)
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018


Since we are looking to analyze the relationship between daily volatility and returns, an additional column that makes sense to add is one which says True when the daily return is positive, and False when the daily return negative. We can then group days into positive and negative return cohorts and compare the average volatility on those days. We will name this column ReturnFlag.

In [12]:
clean_df['ReturnFlag'] = clean_df.apply(lambda row: True if row['Return'] > 0 else False, axis=1) # Volume in Millions
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True


### Exercise 2:

Using ```apply()``` and ```lambda```, write code to create a new column named ```YYYY``` to ```clean_df```, where the new column is the year of the observation as a string. For instance if the row ```Date``` value is 2014-07-28, then the value in the new column for the year would be '2014'. Recall that you can access the first 4 characters of some string ```my_string``` using ```my_string[:4]```.

In [13]:
clean_df['YYYY'] = clean_df.apply(lambda row: row['Date'][:4] ,axis = 1)

### Exercise 3:
Using ```apply()``` and ```lambda```, write a script to create a new column in ```clean_df``` named ```AvgDailyPrice``` that calculates an average daily price based on whether or not the daily volume is over 5 million. Set the value of the new column to (Open + High + Low + Close)/4 if the volume is larger than 5 million, or set the value to (High + Low + Close)/3 if the volume is less than or equal to 5 million.

In [14]:
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014


In [15]:
clean_df['AvgDailyPrice'] =  clean_df.apply(lambda row: (row['Open'] + row['High'] + row['Low']\
                                                         + row['Close'])/4 if row['Volume_Millions'] > 5.0\
                                            else (row['High'] + row['Low'] + row['Close'])/3,axis = 1
                                           )

In [16]:
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014,70.563332
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014,70.280001
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014,69.343336
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014,68.023333
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014,67.740000


### Excercise 4:

Using ```clean_df``` and a ```lambda``` statement within ```apply()```, write a function ```new_custom_function()``` and a script to add a new column to the DataFrame (call it ```EnhancedVolLevel```) that operates in a similar manner as VolLevel but instead gives five volatility level categories using the following logic to determine the label for the volatility level:

```python
if VolStrat > 90th percentile:
    VolLevel = '5_VERY_HIGH'
elif VolStrat > 75th percentile:
    VolLevel = '4_HIGH'
elif VolStrat > 25th percentile:
    VolLevel = '3_MEDIUM'
elif VolStrat > 10th percentile:
    VolLevel = '2_LOW'
else:
    VolLevel = '1_VERY_LOW'
```

Remember each percentile should be calculated by symbol. Use these new labels to see if there are any patterns between volatility levels and the direction of returns. Produce the DataFrame to be able to run the following command:

```python
clean_df.groupby(['Symbol','EnhancedVolLevel'])['ReturnFlag'].mean()
```

In [17]:
Symbol_q = clean_df.groupby('Symbol')['VolStat'].quantile([0.1,0.25,.75,0.9]).to_dict()

In [18]:
def new_custom_function(row):
    
    # Extracting the necessary information
    Symbol_row = row['Symbol']
    VolStrat = row['VolStat']
    # Conditionals
    if VolStrat > Symbol_q[(Symbol_row,0.9)]:
        VolLevel = '5_VERY_HIGH'
    elif VolStrat > Symbol_q[(Symbol_row,0.75)]:
        VolLevel = '4_HIGH'
    elif VolStrat > Symbol_q[(Symbol_row,0.25)]:
        VolLevel = '3_MEDIUM'
    elif VolStrat > Symbol_q[(Symbol_row,0.1)]:
        VolLevel = '2_LOW'
    else:
        VolLevel = '1_VERY_LOW'
        
    return VolLevel
    
    
    

In [19]:
clean_df['EnhancedVolLevel'] = clean_df.apply(lambda row: new_custom_function(row) , axis = 1)

In [20]:
clean_df.groupby(['Symbol','EnhancedVolLevel'])['ReturnFlag'].mean()

Symbol  EnhancedVolLevel
D       1_VERY_LOW          0.508333
        2_LOW               0.573034
        3_MEDIUM            0.558725
        4_HIGH              0.528090
        5_VERY_HIGH         0.408333
DUK     1_VERY_LOW          0.550000
        2_LOW               0.522472
        3_MEDIUM            0.558725
        4_HIGH              0.511236
        5_VERY_HIGH         0.433333
EXC     1_VERY_LOW          0.533333
        2_LOW               0.533708
        3_MEDIUM            0.508389
        4_HIGH              0.533708
        5_VERY_HIGH         0.500000
NEE     1_VERY_LOW          0.566667
        2_LOW               0.550562
        3_MEDIUM            0.567114
        4_HIGH              0.522472
        5_VERY_HIGH         0.500000
SO      1_VERY_LOW          0.516667
        2_LOW               0.511236
        3_MEDIUM            0.530201
        4_HIGH              0.500000
        5_VERY_HIGH         0.516667
Name: ReturnFlag, dtype: float64

In [36]:
dir_SPY = dir_data + 'SPY.csv'

In [37]:
market_df = pd.read_csv(dir_SPY)

In [38]:
market_df['Symbol'] = 'SPY'
market_df['Return'] = (market_df['Close'] / market_df['Open']) - 1.0 
market_df['VolStat'] = (market_df['High'] -  market_df['Low']) /market_df['Open'] 
market_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Return,VolStat
0,2014-08-18,196.800003,197.449997,196.690002,197.360001,178.729111,75424000,SPY,0.002846,0.003862
1,2014-08-19,197.839996,198.539993,197.440002,198.389999,179.661896,59135000,SPY,0.002780,0.005560
2,2014-08-20,198.119995,199.160004,198.080002,198.919998,180.141846,72763000,SPY,0.004038,0.005451
3,2014-08-21,199.089996,199.759995,198.929993,199.500000,180.667160,67791000,SPY,0.002059,0.004169
4,2014-08-22,199.339996,199.690002,198.740005,199.190002,180.386368,76107000,SPY,-0.000752,0.004766


In [39]:
# Merge inner (merge market_df onto clean_df using the dates of clean_df as the keys)
merged_df = pd.merge(clean_df, market_df[['Date','Return']], how='inner', on='Date', suffixes=('','_SPY'))

In [40]:
merged_df.groupby('Symbol')['Date'].count()

Symbol
D      1178
DUK    1178
EXC    1178
NEE    1178
SO     1178
Name: Date, dtype: int64

### Exercise 5:

Using only Symbol D in ```clean_df``` and ```market_df```, use ```pd.merge()``` to determine how many dates are in ```clean_df``` that are not in ```market_df```. Additionally, how many dates are in ```market_df``` that are not in ```clean_df```? (Hint: isnull() may be useful to simplify the solution.)

In [41]:
d_df = pd.DataFrame(clean_df[clean_df['Symbol'] == 'D' ][['Date','Return']])
### Dates in clean_df that are not in market_df
out_date = pd.merge(d_df,market_df[['Date','Return']],how = 'left', on = 'Date',suffixes= ('','_Spy'))
print("Dates only on clean_df not in market_df: ")
print(out_date[out_date.Return_Spy.isna()]['Date'].unique())
print("Number of dates: ", out_date.Return_Spy.isna().sum())

Dates only on clean_df not in market_df: 
['2014-07-28' '2014-07-29' '2014-07-30' '2014-07-31' '2014-08-01'
 '2014-08-04' '2014-08-05' '2014-08-06' '2014-08-08' '2014-08-11'
 '2014-08-12' '2014-08-13' '2014-08-14' '2014-08-15']
Number of dates:  14


In [42]:
### Dates in clean_df that are not in market_df
out_dt = pd.merge(market_df[['Date','Return']],d_df[['Date','Return']],how = 'left', on = 'Date',
                  suffixes=('','_SPY'))
print("Dates only on market_df not in clean_df: ")
print(out_dt[out_dt.Return_SPY.isna()]['Date'].unique())
print("Number of dates: ", out_dt.Return_SPY.isna().sum())

Dates only on market_df not in clean_df: 
['2014-09-19' '2014-10-15' '2014-10-22' '2014-11-04' '2015-02-18'
 '2015-02-24' '2015-02-25' '2015-07-13' '2015-07-14' '2015-07-15'
 '2015-07-29' '2015-07-30' '2015-07-31' '2015-10-14' '2015-12-21'
 '2016-01-14' '2016-01-15' '2016-01-19' '2016-01-20' '2016-04-04'
 '2016-04-26' '2016-09-22' '2017-06-07' '2017-07-28' '2017-07-31'
 '2017-08-17' '2017-08-25' '2017-09-05' '2018-05-14' '2018-05-15'
 '2018-05-16' '2018-05-17' '2018-05-18' '2018-05-21' '2018-05-22'
 '2018-05-23' '2018-05-24' '2018-05-25' '2018-06-07' '2018-09-20'
 '2018-09-21' '2018-09-24' '2018-09-25' '2018-09-26' '2018-09-27'
 '2018-09-28' '2018-10-01' '2018-10-02' '2018-10-03' '2018-10-04'
 '2018-10-05' '2018-10-08' '2018-10-09' '2018-10-18' '2018-10-19'
 '2019-04-02' '2019-04-03' '2019-04-04' '2019-04-05' '2019-04-08'
 '2019-04-09' '2019-04-10' '2019-04-17' '2019-07-01' '2019-07-02'
 '2019-07-16' '2019-07-29' '2019-07-30' '2019-07-31' '2019-08-01'
 '2019-08-02' '2019-08-05' '2019-0

### Exercise 7:

Use ```market_df``` and ```clean_df``` to create a new DataFrame ```modified_clean_df``` that is the same as ```clean_df``` but with a new column named ```MeanMonthSPYReturn```. Each value in the new column should be the mean monthly return for SPY for the given month of each row's ```Date```. The output for the first and last rows of the ```modified_clean_df``` DataFrame should produce the following output from head(1) and tail(1):
```python
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].head(1))
```

                 Date       Symbol YYYYMM MeanMonthSPYReturn
                 2014-08-01 D      201408 0.001443

```python
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].tail(1))
```
                 Date       Symbol YYYYMM MeanMonthSPYReturn
                 2019-07-26 DUK    201907 0.000167

In [51]:
def date_month(row):
    return row['Date'][:4] + row['Date'][5:7]


In [57]:
market_df['YYYYMM'] = market_df.apply(lambda row: date_month(row), axis = 1)
clean_df['YYYYMM'] = clean_df.apply(lambda row : date_month(row), axis = 1)

In [58]:
market_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Return,VolStat,YYYYMM
0,2014-08-18,196.800003,197.449997,196.690002,197.360001,178.729111,75424000,SPY,0.002846,0.003862,201408
1,2014-08-19,197.839996,198.539993,197.440002,198.389999,179.661896,59135000,SPY,0.002780,0.005560,201408
2,2014-08-20,198.119995,199.160004,198.080002,198.919998,180.141846,72763000,SPY,0.004038,0.005451,201408
3,2014-08-21,199.089996,199.759995,198.929993,199.500000,180.667160,67791000,SPY,0.002059,0.004169,201408
4,2014-08-22,199.339996,199.690002,198.740005,199.190002,180.386368,76107000,SPY,-0.000752,0.004766,201408


In [65]:
## Grouping by YYYYMM
market_df_grp = market_df.groupby('YYYYMM')['Return'].mean().reset_index().rename(columns = {'Return':'MeanMonthSPYReturn' })    
market_df_grp.head()

,YYYYMM,MeanMonthSPYReturn
0,201408,0.001443
1,201409,-0.000285
2,201410,0.000407
3,201411,0.000920
4,201412,-0.000325


In [69]:
modified_clean_df = pd.merge(clean_df,market_df_grp, how = "inner", on = "YYYYMM")

In [75]:
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].head(1))
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].tail(1))

         Date Symbol  YYYYMM  MeanMonthSPYReturn
0  2014-08-01      D  201408            0.001443
            Date Symbol  YYYYMM  MeanMonthSPYReturn
5939  2019-07-26    DUK  201907            0.000167


## Breaking down returns based on the broader market return
Now we can begin our granular analysis of how broader market returns are related to single stock returns. 

Let's begin by breaking down broader market returns into quantiles. This quantile analysis approach we will take is commonly employed in data analysis to determine how the magnitude of one variable is related to another variable of interest.

We can explore this idea with the ```pd.qcut()``` method. Namely, ```pd.qcut()``` will allow us to cut the market returns by quantiles (we will eventually group by quantiles), and therefore will allow us to calculate summary statistics (such as average return) for each quantile.

Let's first extract the returns using the convenience of the ```pivot()``` method on a DataFrame. Pivoting a DataFrame may be accomplished by specifying:

1. An index to pivot on. In this case we choose ```Date```.
2. Columns that we'd like to have after the pivot. In this case we'd like columns that are the ````Symbol```.
3. The values that each (row,column) pair will show. In this case we'd like to have the ```Return```.

We will pivot ```merged_df``` using these parameter inputs to output a DataFrame where the rows are the Date, each column is the Symbol, and the values are the open-to-close daily return for the given date and symbol.

In [77]:
# Extract returns from merged_df, where we use pivot to simplify the task
return_df = merged_df.pivot(index='Date', columns='Symbol', values='Return')
return_df.head()

Symbol,D,DUK,EXC,NEE,SO
Date,,,,,
2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446
2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741
2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508
2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000
2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195


Let's merge on the broader market returns from the SPY ```market_df``` loaded earlier:

In [81]:
# Let's merge the SPY (broader market) returns by Date onto the return_df DataFrame
full_df = pd.merge(return_df, market_df[['Date','Return']].set_index('Date'), left_index=True, right_index=True)
full_df = full_df.rename(columns={'Return':'MarketReturn'}).reset_index()
full_df.head()

,Date,D,DUK,EXC,NEE,SO,MarketReturn
0,2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446,0.002846
1,2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741,0.002780
2,2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508,0.004038
3,2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000,0.002059
4,2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195,-0.000752


In [83]:
# Create 10 quantile categories by the market return
num_quantiles = 10
full_df['market_quantile'] = pd.qcut(full_df['MarketReturn'],num_quantiles,labels=False)
full_df.head()

,Date,D,DUK,EXC,NEE,SO,MarketReturn,market_quantile
0,2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446,0.002846,7
1,2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741,0.002780,7
2,2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508,0.004038,7
3,2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000,0.002059,6
4,2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195,-0.000752,3


In [85]:
# Merge market volstat onto returns
new_df = pd.merge(return_df, market_df[['Date','VolStat']].set_index('Date'), left_index=True, right_index=True)
new_df = new_df.rename(columns={'VolStat':'MarketVolStat'}).reset_index()

# Add on quantiles
num_quantiles = 10
new_df['market_volstat_quantile'] = pd.qcut(new_df['MarketVolStat'],num_quantiles,labels=False)

# Calcualte mean by quantile
new_df.groupby('market_volstat_quantile').std()

,D,DUK,EXC,NEE,SO,MarketVolStat
market_volstat_quantile,,,,,,
0,0.007394,0.007464,0.009186,0.007310,0.008521,0.000508
1,0.007177,0.007135,0.009354,0.007658,0.006974,0.000262
2,0.008480,0.008084,0.009745,0.007748,0.007407,0.000242
3,0.008298,0.008194,0.010211,0.008106,0.007824,0.000245
4,0.009563,0.008093,0.010409,0.007003,0.008374,0.000301
5,0.009868,0.008642,0.010350,0.008032,0.008836,0.000370
6,0.008900,0.009298,0.013011,0.008751,0.009049,0.000509
7,0.009557,0.009409,0.010587,0.010960,0.008547,0.000674
8,0.010907,0.011419,0.013512,0.010837,0.010354,0.001292
